# ALGOS DE RECO #

In [1]:
# on importe les differentes librairies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix

# on importe les différentes librairies surprise de scikit
from surprise import SVD
from surprise import dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.knns import KNNBasic, KNNWithMeans

# on importe notre bdd
avis = pd.read_csv("BDD/avis.csv", index_col="Unnamed: 0")
#jeux = pd.read_csv("BDD/jeux.csv", index_col="Unnamed: 0")

C:\Users\hacho\AppData\Local\Temp\ipykernel_20796\868762237.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
avis.head(1)

,author,date_published,title_review,note,title,url,comment
0,Monsieur Guillaume,2021-01,Voyages sur les ailes des papillons,8,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,"Lorsque le jeu est jeu, bon, réflexif, joli po..."


Il faut mettre en place la matrice sparse (calculer la sparsité + plot cette sparsite)

In [3]:
num_users  = avis["author"].nunique()
num_items  = avis["title"].nunique()

print(f"Il y'a {num_users} auteurs et {num_items} jeux")

sparsity = (len(avis) / (num_users*num_items))*100
print(f"Notre matrice a une sparsité de {sparsity:.2f}% ")

matrice_sparse = csr_matrix(avis['note'])
matrice_sparse

Il y'a 2564 auteurs et 3057 jeux
Notre matrice a une sparsité de 1.60% 


<1x125299 sparse matrix of type '<class 'numpy.int64'>'
	with 124925 stored elements in Compressed Sparse Row format>

On test maintenant les differents algos (SVD + KNN pour commencer)

## SVD ##

In [4]:
algo = SVD()

reader = Reader(line_format='user item rating', rating_scale=(0,10))

class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, jeux, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(avis['author'], avis['title'], avis['note'])]
        self.reader=reader

data = MyDataset(avis, reader)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.8703  1.8811  1.8632  1.8664  1.8669  1.8696  0.0062  
MAE (testset)     1.4366  1.4443  1.4297  1.4353  1.4343  1.4360  0.0047  
Fit time          1.00    0.97    0.98    0.95    0.98    0.98    0.02    
Test time         0.11    0.16    0.11    0.16    0.10    0.13    0.03    


{'test_rmse': array([1.8702545 , 1.88111702, 1.86316642, 1.86644486, 1.86686568]),
 'test_mae': array([1.43659505, 1.44428263, 1.42973383, 1.43525891, 1.43427045]),
 'fit_time': (1.0042600631713867,
  0.9660162925720215,
  0.9824082851409912,
  0.9496316909790039,
  0.9774348735809326),
 'test_time': (0.10529875755310059,
  0.1609199047088623,
  0.10578536987304688,
  0.16041994094848633,
  0.09834003448486328)}

## k-NN ##

In [5]:
algo = KNNBasic()

reader = Reader(line_format='user item rating', rating_scale=(0,10))

class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, jeux, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(avis['author'], avis['title'], avis['note'])]
        self.reader=reader

data = MyDataset(avis, reader)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.9173  1.9099  1.9158  1.9154  1.9141  1.9145  0.0025  
MAE (testset)     1.4863  1.4804  1.4843  1.4858  1.4822  1.4838  0.0022  
Fit time          0.50    0.50    0.50    0.53    0.51    0.51    0.01    
Test time         1.21    1.28    1.20    1.28    1.21    1.24    0.04    


{'test_rmse': array([1.91729947, 1.90990949, 1.91581034, 1.91538572, 1.91410828]),
 'test_mae': array([1.48634277, 1.48037957, 1.48433865, 1.48578265, 1.48219897]),
 'fit_time': (0.495638370513916,
  0.5021712779998779,
  0.49783968925476074,
  0.5282132625579834,
  0.5135533809661865),
 'test_time': (1.2108161449432373,
  1.2781243324279785,
  1.2010433673858643,
  1.2843799591064453,
  1.2093863487243652)}

In [12]:
avis_par_jeux = avis.groupby('title').size()

avis_par_jeux.to_dict()

{'1, 2 Truie !': 13,
 "10' to kill": 99,
 '11 nimmt': 10,
 '13 jours, la crise des missiles de Cuba, 1962': 17,
 '1775 - la révolution américaine': 34,
 "1812 - L'Invasion du Canada": 30,
 '1830': 10,
 '1944 : Race to the Rhine': 16,
 '1960 : Kennedy contre Nixon': 112,
 '1969': 12,
 '2 de Mayo': 17,
 '2 sans 3': 12,
 '20ème Siècle': 23,
 '22 Pommes': 49,
 '2eme DB - Normandie': 11,
 '3, 2, 1... Couleurs !': 10,
 '30 Carats': 54,
 '5 Cornichons': 13,
 '51st State - Seconde édition française': 10,
 '5211': 18,
 '6 qui prend !': 150,
 '6 qui surprend !': 37,
 '7 Wonders - Extension "Armada"': 42,
 '7 Wonders - Extension "Babel"': 55,
 '7 Wonders - Extension "Cities"': 135,
 '7 Wonders - Extension "Wonder Pack"': 50,
 '7 Wonders : Louis Armstrong': 13,
 '7 Wonders : Manneken Pis': 20,
 '7 Wonders : Stevie Wonders': 16,
 '7 Wonders : Île de Catane': 33,
 '7 Wonders Duel': 148,
 '7 Wonders Duel - Agora': 22,
 '7 Wonders Duel - Extension "Panthéon"': 94,
 '7 Wonders Duel : Messe': 16,
 '7 Wo

In [27]:
jeux_tries = sorted(avis_par_jeux.items(), key=lambda x: x[1], reverse=True)
jeux_tries

top_10_jeux = jeux_tries[:10]
top_10_jeux

[('Non Merci', 159),
 ('Shogun', 157),
 ('Camelot', 152),
 ('6 qui prend !', 150),
 ('Agricola', 150),
 ('Bang!', 150),
 ('Blokus to go', 150),
 ('Bohnanza', 150),
 ("Ca$h'n Guns second édition", 150),
 ("Can't Stop", 150)]

In [37]:
top_10_jeux_df = pd.DataFrame.from_dict(top_10_jeux)
top_10_jeux_df.rename(columns={0: 'title', 1:'nb avis'})

,title,nb avis
0,Non Merci,159
1,Shogun,157
2,Camelot,152
3,6 qui prend !,150
4,Agricola,150
5,Bang!,150
6,Blokus to go,150
7,Bohnanza,150
8,Ca$h'n Guns second édition,150
9,Can't Stop,150


In [41]:
comments_by_title = avis.groupby('title')['comment'].unique()
comments_by_title_df = pd.DataFrame(comments_by_title)
comments_by_title_df

,comment
title,
"1, 2 Truie !",[Un petit jeu de défausse avec un peu de mathé...
10' to kill,[Les + : \nJeu simple et accessible à tous\nPa...
11 nimmt,[Les + :Règles simples et vite expliquéesJouab...
"13 jours, la crise des missiles de Cuba, 1962",[SI Twilight truggle est la réference du w\tar...
1775 - la révolution américaine,"[nan, Meilleur wargame de 2013 ? Alors là.... ..."
...,...
À la Carte,[J'ai eu le plaisir de faire une partie de l'a...
À la gloire d'Odin,"[La boîte pèse 15 kg, coûte 90 e !!!, (filosof..."
À vos marques,"[Attention aux jeunes mains fragiles, à force ..."
